## 5. Stock.

### 5.1 Creation of the csv of the stock with the predictions of the first 4 weeks.

In [ ]:
df_stock = df_con_predicciones[df_con_predicciones['date'] > '2016-04-24']

In [ ]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120960 entries, 8285760 to 8406719
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          120960 non-null  object
 1   date        120960 non-null  object
 2   item        120960 non-null  object
 3   category    120960 non-null  object
 4   store_code  120960 non-null  object
 5   region      120960 non-null  object
 6   department  120960 non-null  object
 7   n_sales     120960 non-null  object
 8   sell_price  120960 non-null  object
 9   revenue     120960 non-null  object
dtypes: object(10)
memory usage: 10.2+ MB


In [ ]:
df_stock.head()

,id,date,item,category,store_code,region,department,n_sales,sell_price,revenue
8285760,ACCESORIES_1_001_BOS_1,2016-05-01,ACCESORIES_1_001,ACCESORIES,BOS_1,Boston,ACCESORIES_1,"2,69","10,99","29,57"
8285761,ACCESORIES_1_001_BOS_2,2016-05-01,ACCESORIES_1_001,ACCESORIES,BOS_2,Boston,ACCESORIES_1,"3,75","10,99","41,22"
8285762,ACCESORIES_1_001_BOS_3,2016-05-01,ACCESORIES_1_001,ACCESORIES,BOS_3,Boston,ACCESORIES_1,"3,93","10,99","43,23"
8285763,ACCESORIES_1_001_NYC_1,2016-05-01,ACCESORIES_1_001,ACCESORIES,NYC_1,New York,ACCESORIES_1,"6,10","10,99","66,97"
8285764,ACCESORIES_1_001_NYC_2,2016-05-01,ACCESORIES_1_001,ACCESORIES,NYC_2,New York,ACCESORIES_1,"6,21","10,99","68,17"


### 5.2 Implementation of the EOQ model.

#### The EOQ (Economic Order Quantity) model is a widely used tool for stock control and shop supply. This model allows us to make a good control both in optimising the order size and the safety stock that we have to have in each of our stores depending on each ID. In fact, this model is very useful when dealing with products where there is a high variability in the number of sales, from products that sell very little to others that sell assiduously.

In order to apply the formula on which this model is based, we need to have certain variables that we can find:
  - Average sales calculation (D): in our case it will be the value of the column df_stock['n_sales'].
  - Standard deviation of product demand (σ): a new column will be created below to show this deviation, df_stock['std_desviation'].
  - Cost of placing an order(S): a column expressing the cost of placing an order for each product, in our case we chose 10% of the sell_price, although the logic team could give us an estimate. This variable will therefore be df_stock['sell_price']*0.10.
  - Cost of holding a product (H): variable corresponding to what it costs the company to hold a product in stock, we have chosen 5%, although, as in the case of variable S, logistics could adjust this value. This variable corresponds to df_stock['sell_price']*0.05.


In [ ]:
df_stock['sell_price'] = df_stock['sell_price'].str.replace(',','.')

In [ ]:
df_stock['sell_price'] = df_stock['sell_price'].astype(float)

In [ ]:
df_stock['n_sales'] = df_stock['n_sales'].str.replace(',','.')

In [ ]:
df_stock['n_sales'] = df_stock['n_sales'].astype(float)

In [ ]:
df_stock['n_sales'] = df_stock['n_sales'].apply(lambda x: 0 if x < 0 else x)

In [ ]:
std = df_stock.groupby('id')['n_sales'].std()

In [ ]:
df_stock['std_desviation'] = df_stock['id'].map(std)

#### We also create the variable week to make it easier to make our queries later on.

In [ ]:
df_stock['date'] = pd.to_datetime(df_stock['date'])

In [ ]:
df_stock['week']= df_stock['date'].dt.isocalendar().week

In [ ]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120960 entries, 8285760 to 8406719
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              120960 non-null  object        
 1   date            120960 non-null  datetime64[ns]
 2   item            120960 non-null  object        
 3   category        120960 non-null  object        
 4   store_code      120960 non-null  object        
 5   region          120960 non-null  object        
 6   department      120960 non-null  object        
 7   n_sales         120960 non-null  float64       
 8   sell_price      120960 non-null  float64       
 9   revenue         120960 non-null  object        
 10  std_desviation  120960 non-null  float64       
 11  week            120960 non-null  UInt32        
dtypes: UInt32(1), datetime64[ns](1), float64(3), object(7)
memory usage: 11.7+ MB


In [ ]:
df_stock['H'] = df_stock['sell_price'] * 0.05
df_stock['S'] = df_stock['sell_price'] * 0.10

#### 5.2.1 Economic Order Quantity (EOQ).
We start by calculating the Economic Order Quantity (EOQ), which is the number of units when ordering one of our products. In our case, the variable D must be adjusted to 4 weeks by multiplying it by 4:
  1. D = D x 4.
  2. EOQ = √(2DS/H)

We store the value in the column -> df_stock['n_products_ordered'].

In [ ]:
df_stock['D_4'] = df_stock['n_sales'] * 4

df_stock['n_productos_pedido'] = np.sqrt((2 * df_stock['D_4'] * df_stock['S']) / df_stock['H'])

Calculate Safety Stock (SS) The Safety Stock (SS) depends on the variability in demand and the desired service level (probability of not having shortages): S S = Z × σ × L SS=Z×σ× L where: Z = Service level factor (based on the normal distribution, e.g. 1.65 for a service level of 95%). σ = Standard deviation of the weekly demand. L = Lead time in weeks.



#### 5.2.2 Safety Stock(SS).
Here we calculate the number of inventory units used to mitigate demand variance. This means that it is the extra stock that is held to cope with unforeseen events related to changes in demand or supply problems from suppliers. To do this, we need to implement the variables:
  - Z = service level factor, which in a normal distribution would be 1.65 and is the value we will use.
  - L = lead time, in our case 1 week, i.e. 1.

The formula for calculating this stock is therefore
  - SS = Z x σ x L.

We store the value in the column -> df_stock['stock_extra'].

In [ ]:
Z = 1.65

df_stock['stock_extra'] = Z * df_stock['std_desviation'] * np.sqrt(1)

#### 5.2.3 Reorder Point (ROP).
Represents the inventory level at which a new order must be placed to replenish the stock before it runs out. Its formula is:
  - ROP = (D x L) + SS.

We are going to estimate it as the minimum accurate stock to not have to reorder that product.

We store it in the variable df_stock[‘stock_preciso’].

In [ ]:
df_stock['stock_preciso'] = (df_stock['n_sales'] * 1) + df_stock['stock_extra']

In [ ]:
df_stock.head()

,id,date,item,category,store_code,region,department,n_sales,sell_price,revenue,std_desviation,week,H,S,D_4,n_productos_pedido,stock_extra,stock_preciso
8285760,ACCESORIES_1_001_BOS_1,2016-05-01,ACCESORIES_1_001,ACCESORIES,BOS_1,Boston,ACCESORIES_1,2.69,10.99,"29,57",0.29,17,0.55,1.10,10.76,6.56,0.48,3.17
8285761,ACCESORIES_1_001_BOS_2,2016-05-01,ACCESORIES_1_001,ACCESORIES,BOS_2,Boston,ACCESORIES_1,3.75,10.99,"41,22",0.32,17,0.55,1.10,15.00,7.75,0.52,4.27
8285762,ACCESORIES_1_001_BOS_3,2016-05-01,ACCESORIES_1_001,ACCESORIES,BOS_3,Boston,ACCESORIES_1,3.93,10.99,"43,23",0.23,17,0.55,1.10,15.72,7.93,0.38,4.31
8285763,ACCESORIES_1_001_NYC_1,2016-05-01,ACCESORIES_1_001,ACCESORIES,NYC_1,New York,ACCESORIES_1,6.10,10.99,"66,97",0.69,17,0.55,1.10,24.40,9.88,1.15,7.25
8285764,ACCESORIES_1_001_NYC_2,2016-05-01,ACCESORIES_1_001,ACCESORIES,NYC_2,New York,ACCESORIES_1,6.21,10.99,"68,17",0.65,17,0.55,1.10,24.84,9.97,1.07,7.28


#### Once we have created all our new variables on which we will base our stock, it would be necessary to adjust the formulas depending on the type of product, such as those products that are more perishable.

In [ ]:
lista_redondeo = ['n_productos_pedido','n_sales', 'stock_extra', 'stock_preciso']
for i in lista_redondeo:
  df_stock[i] = np.ceil(df_stock[i])
  df_stock[i] = df_stock[i].astype(int)

In [ ]:
df_stock['week'] = df_stock['week'].astype(object)

In [ ]:
df_stock = df_stock[['id', 'week', 'n_sales', 'n_productos_pedido', 'stock_extra', 'stock_preciso']]

#### All those products where the stock_precise is 0, will be products that customers will be able to buy on demand.

In [ ]:
df_stock.describe()

,n_sales,n_productos_pedido,stock_extra,stock_preciso
count,"120,960.00","120,960.00","120,960.00","120,960.00"
mean,10.24,10.77,2.67,12.41
std,20.02,7.10,4.11,22.83
min,0.00,0.00,0.00,0.00
25%,3.00,6.00,1.00,3.00
50%,5.00,9.00,2.00,6.00
75%,10.00,13.00,3.00,13.00
max,705.00,107.00,175.00,705.00


In [ ]:
df_stock.describe(include='object')

,id,week
count,120960,120960
unique,30240,4
top,SUPERMARKET_3_824_BOS_1,17
freq,4,30240


#### We create our final csv document with the stock for the 4 weeks, which we will also use to generate an API and query its values from it.

In [ ]:
df_stock.to_csv('df_stock.csv', sep = ';', index=False)